<h1><b>ETL DADOS - WELLBE</b></h1>

<p><b>A linha de raciocínio seguida foi a seguinte:</b></p>

<ul>
    <li>Obter os dados vindos do arquivo fornecido</li>
    <li>Extrair os dados utilizando pandas</li>
    <li>Passar os dados extraídos para um dataframe (df)</li>
    <li>Tratar os dados com pandas e numpy</li>
    <ul>
        <li>Algumas colunas possuem valores vazios/diferentes dos valores padrão</li>
        <li>Localizar o departamento caso esteja faltando</li>
        <li>Remover as colunas que não têm dados</li>
        <li>Conferir se todos os dados da respectiva coluna estão no mesmo formato (texto, número)</li>
        <li>Verificar se os valores de custo fazem sentido </li>
        <li>Deixar o df padronizado</li>
    </ul>
    <li>Escrever os dados tratados no banco de dados</li>
</ul>

<h2>EXTRACT - Extraindo os dados da tabela fornecida</h2>

<p>Importando as ferramentas necessárias</p>

In [2]:
import pandas as pd
import mysql.connector
import numpy as np
import dotenv
import os

<h3>Lendo a planilha a partir da linha que contém os dados</h3>

In [3]:
df = pd.read_excel("Dados_Desafio.xlsx", skiprows=13, engine="openpyxl")

<h3>Renomeando as colunas</h3>

In [4]:
df.columns = ["codigo", "custo", "identificacao", "funcionario", "departamento", "data_atestado", "especialidade", "motivo", "lider"]

<h2>TRANSFORM - Tratando os dados extraídos no df</h2>

<h3>Removendo as colunas que estão vazias</h3>

In [5]:
df.dropna(axis=1, how='all', inplace=True)

<h3>Removendo os espaços presentes da coluna e renomeando os valores vazios</h3>

In [6]:
df["departamento"] = df["departamento"].astype(str).str.strip
df["departamento"] = df["departamento"].replace(["", "nan", "None"], np.nan)

<h3>Conferindo se é possível deduzir o departamento caso esteja faltando</h3>

<p>Observou-se que alguns funcionários estavam sem seu departamento, com isso, a seguinte função (preencher_departamento) tem como finalidade descobrir se o departamento está vazio, em seguida olha para o funcionário anterior/posterior e verifica se ambos tem o mesmo nome, em caso positivo, atribui-se o mesmo departamento, caso contrário, aplica-se o valor 'DESCONHECIDO'</p>

In [7]:
def preencher_departamento(df):
    for idx in range(len(df)):
        if pd.isna(df.at[idx, "departamento"]) or df.at[idx, "departamento"] == "":
            # Verificar a linha acima (se existir)
            if idx > 0 and df.at[idx, "funcionario"] == df.at[idx - 1, "funcionario"]:
                df.at[idx, "departamento"] = df.at[idx - 1, "departamento"]
            # Verificar a linha abaixo (se existir)
            elif idx < len(df) - 1 and df.at[idx, "funcionario"] == df.at[idx + 1, "funcionario"]:
                df.at[idx, "departamento"] = df.at[idx + 1, "departamento"]
            # Se não encontrar, mantém "DESCONHECIDO"
            else:
                df.at[idx, "departamento"] = "DESCONHECIDO"
    return df

df = preencher_departamento(df)

<h3>Convertendo a coluna "data_atestado" para o formato de data</h3>

In [8]:
df["data_atestado"] = pd.to_datetime(df["data_atestado"], dayfirst=True, errors='coerce')

<h3>Tratamento da coluna "custo" (alguns itens possuem "--" ou são itens vazios, esses serão substituídos por None, que representa o Null em SQL</h3>

<p>Esse tratamento utiliza a função corrigir_custo, implementada também para substituir a "," por "." nas casas decimais e converter para número</p>

In [9]:
def corrigir_custo(valor):
    if pd.isna(valor) or valor == "--":
        return None  
    try:
        return float(str(valor).replace(",", "."))  
    except ValueError:
        return None  

df["custo"] = df["custo"].apply(corrigir_custo)

<h3>Preenchendo os valores vazios presentes nas colunas de texto com "Desconhecido"</h3>

In [10]:
df.fillna({"especialidade": "Desconhecido", "motivo": "Desconhecido"}, inplace=True)

<h3>Remove as linhas do dataframe que não possuam código</h3>
<p>O código do atestado é a identificação primária dos dados, caso não possua, o erro é tratado</p>

In [11]:
df = df.dropna(subset=["codigo"])

<h2>LOAD - Carregando os dados tratados no banco de dados</h2>

<h3>Estabelecendo a conexão com o banco de dados local</h3>

In [14]:
dotenv.load_dotenv()
conn = mysql.connector.connect(
    host=os.environ['HOST_DB'],
    user=os.environ['USER_DB'],  
    password=os.environ['PASSWORD_DB'],  
    database=os.environ['DATABASE']
)
cursor = conn.cursor()

<h3>Inserindo os dados no banco</h3>
<p>Antes de inserir no banco, os valores "NaN" (Not a Number) presentes, foram convertidos para "None", que o MySQL reconhece como "Null"</p>
<p>O banco é primeiramente limpo, caso já existam dados</p>

In [13]:
try:
    cursor.execute("DELETE FROM atestados")
    cursor.execute("ALTER TABLE atestados AUTO_INCREMENT = 1")
    
    for _, row in df.iterrows():
        sql = """
            INSERT INTO atestados (codigo, custo, funcionario, departamento, data_atestado, especialidade, motivo, lider)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        valores = (
            row["codigo"], row["custo"], row["funcionario"], row["departamento"],
            row["data_atestado"], row["especialidade"], row["motivo"], row["lider"]
        )
        valores = [None if isinstance(v, float) and np.isnan(v) else v for v in valores]
        cursor.execute(sql, valores)
    
    conn.commit()

except Exception as e:
    print(f'Erro: {e}')
    conn.rollback()

finally:
    cursor.close()
    conn.close()

Erro: Python type method cannot be converted
